# Project Discrete Optimization and Decision Making

## First point

This project involves designing an efficient last-mile delivery system for a city. A company has a set of customers $C = \{1, \ldots, \overline{c}\}$, each requiring the delivery of a package. The weight of each package $w_c, c \in C$ is known. The delivery time for each customer is equal to $s_c, c \in C$. The company has a set $K = \{1, \ldots, \overline{k}\}$ of delivery vans available that all start from the same depot $0$ at time $0$, have the same capacity $W$, and must return to the depot withing $t_{max}$. Each vehicle can exit the depot at most once. The problem can be formulated on a complete directed graph $G = (V, A)$ where $V = C \cup \{0\}$ is the set of nodes and $A = \{(i,j) | i, j \in V, i \neq j\}$ is the set of arcs. Each arc can be traveled at most once by any of the vehicles. For each arc $(i, j) \in A$, let us define the time $t_{ij}$ required to travel over the arc. Travel times satisfy the triangle inequality. The goal of the company is to minimize the total time required to complete the service for all customers. Provide a mathematical formulation of the problem and its optimal solution. 

### Mathematical Model

## Import Libraries

In [48]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
import random
import gurobipy as gp
from gurobipy import GRB

## First point

In [75]:
import gurobipy as gp
from gurobipy import GRB

# Sample data
N_0 = [1, 2, 3]  # Customers
K = [1,2]     # Vehicles
N = N_0 + [0]    # Nodes (including depot 0)
A = [(i, j) for i in N for j in N if i != j]
Q = 10         # Vehicle capacity
q = {1: 2, 2: 3, 3: 4}  # Package weights
t = {0: 0, 1: 40, 2: 20, 3: 10}  # Service times
# Travel times
t_ij = {
    (0, 1): 20, (0, 2): 20, (0, 3): 20,
    (1, 0): 20, (1, 2): 25, (1, 3): 30,
    (2, 0): 40, (2, 1): 25, (2, 3): 35,
    (3, 0): 30, (3, 1): 30, (3, 2): 35
}

In [76]:
model = gp.Model("last_mile_delivery_with_time_windows_and_restrictions")
x = model.addVars(A, vtype=GRB.BINARY, name="x")
f = model.addVars(A, vtype=GRB.INTEGER, name = "f")
y = model.addVars(N_0, vtype=GRB.INTEGER, name = "y")
s = model.addVars(N_0, vtype = GRB.INTEGER, name="s")

model.setObjective(gp.quicksum((t_ij[i, j]) * x[i, j] for (i, j) in A), GRB.MINIMIZE)


# (5)
constraint_vehicles = None
for j in N_0:
    if constraint_vehicles == None:
        constraint_vehicles = x[0, j]
    else:
        constraint_vehicles += x[0, j]
model.addConstr(constraint_vehicles == len(K))

# (6)
for i in N_0:
    constraint_entering_flow = None
    for j in N:
        if i != j:
            if constraint_entering_flow == None:
                constraint_entering_flow = x[i, j]
            else:
                constraint_entering_flow += x[i, j]
    model.addConstr(constraint_entering_flow == 1)
# (7)
for j in N_0:
    constraint_exit_flow = None
    for i in N:
        if i != j:
            if constraint_exit_flow == None:
                constraint_exit_flow = x[i, j]
            else:
                constraint_exit_flow += x[i, j]
    model.addConstr(constraint_exit_flow == 1)

# (8)
for i in N_0:
    constraint_quantity_stability = None
    sum_fji = None
    sum_fij = None
    for j in N:
        if i != j:
            if sum_fji == None:
                sum_fji = f[j, i]
            else:
                sum_fji += f[j, i]
            if sum_fij == None:
                sum_fij = f[i, j]
            else:
                sum_fij += f[i, j]
    model.addConstr((sum_fji - sum_fij) == q[i])

# (9)
for (i,j) in A:
    if j != 0:
        model.addConstr((q[j] * x[i, j]) <= f[i, j])
    if i != 0:
        model.addConstr(f[i, j] <= (Q - q[i])*x[i, j])

# (11)
M = 10e7
for (i, j) in A:
    if i != 0 and j != 0:
        model.addConstr(y[i] - y[j] + t[i] + t_ij[i, j] <= M*(1 - x[i, j]))
    elif j != 0:
        model.addConstr(-y[j] + t[i] + t_ij[i, j] <= M*(1 - x[i, j]))

L = 10e7
for j in N_0:
    model.addConstr(y[j] + t[j] - s[j]  <= L*(1 - x[j, 0]))


In [77]:
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    solutions = model.getAttr('x', x)
    for key in solutions:
        if solutions[key] > 0.5:
            print(key, "->",solutions[key])
    model.write('model.lp')
    for v in model.getVars():
        if 'y' in  v.VarName :
            print(f"{v.VarName} = {v.X}")
        if 's' in v.VarName:
            print(f"{v.VarName} = {v.X}")
if model.Status == GRB.INFEASIBLE:
        model.computeIIS()
        model.write('model.ilp')

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 40 rows, 30 columns and 108 nonzeros
Model fingerprint: 0x08240717
Variable types: 0 continuous, 30 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+08]
  Objective range  [2e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+08]
Found heuristic solution: objective 130.0000000
Presolve removed 32 rows and 24 columns
Presolve time: 0.00s
Presolved: 8 rows, 6 columns, 20 nonzeros
Variable types: 0 continuous, 6 integer (3 binary)
Found heuristic solution: objective 115.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 115 130 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.150000000000e+02, best bound 1.150000000000e+02, gap 0